# La ultima Poutraaaaaaaaaaaaaa

In [ ]:
import csv

with open("dataset/train.csv") as file:
  header_and_data = list(csv.reader(file, delimiter=','))

header = header_and_data[0]
data = header_and_data[1:]

print(data)

In [3]:
print(5)

print(7)

print("Yellow World")

print("Coucou de la Poutraaa ")

print("🐍🔥")

5
7
Yellow World
Coucou de la Poutraaa 
